# Calculating the NEX/county intersection grid as well as the growing area within each county

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
# import geopandas as gp
# from shapely.geometry import *
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import xarray as xr

In [2]:
# NEX data 
gdd = xr.open_dataset("../../../data/netCDF/GDD_10-29C_gs_GMFD.nc")
kdd = xr.open_dataset("../../../data/netCDF/GDD_29C_gs_GMFD.nc")
prcp = xr.open_dataset("../../../data/netCDF/cuml_precip_gs_GMFD.nc")

In [ ]:
nex.isel(lat = 400, lon = 1040, time = 50)

## Calculate grid

In [ ]:
nex.isel(lon = 1040)

In [ ]:
# GMFD grid
lat = nex.coords["lat"].values
lat_step = abs(lat[1] - lat[0])

lon = nex.coords["lon"].values
lon_step = abs(lon[1] - lon[0])

US_lat = np.intersect1d(np.argwhere(lat > 24),np.argwhere(lat < 50))
US_lon = np.intersect1d(np.argwhere(lon > 360-125),np.argwhere(lon < 360-66))

In [ ]:
# US lat/lon GeoFrame 
US_geoF = gp.GeoSeries([Point(lon[US_lon[x]]-360, lat[US_lat[y]]) for x in range(len(US_lon)) for y in range(len(US_lat))])
US_geoF = gp.GeoSeries([Polygon([(lon[US_lon[x]]-360 + lon_step/2, lat[US_lat[y]] + lat_step/2),
                                       (lon[US_lon[x]]-360 + lon_step/2, lat[US_lat[y]] - lat_step/2),
                                       (lon[US_lon[x]]-360 - lon_step/2, lat[US_lat[y]] + lat_step/2),
                                       (lon[US_lon[x]]-360 - lon_step/2, lat[US_lat[y]] - lat_step/2)])
                               for x in range(len(US_lon)) for y in range(len(US_lat))])

US_geo_lon = gp.GeoSeries([US_lon[x] for x in range(len(US_lon)) for y in range(len(US_lat))])
US_geo_lat = gp.GeoSeries([US_lat[y] for x in range(len(US_lon)) for y in range(len(US_lat))])

US_geoF = gp.GeoDataFrame({"geometry": US_geoF, "latitude": US_geo_lat, "longitude": US_geo_lon})
#US_geoF["geometry"] = US_geoF.geometry.buffer(d)
US_geoF["geometry"] = US_geoF.geometry.envelope

US_geoF.crs = {'init' :'epsg:4269'}

In [ ]:
# Read in county data
counties = gp.read_file("../../../large_files/contig_county/counties_contig.shp")

In [ ]:
# Overlap
final = gp.overlay(counties, US_geoF, how="intersection")

In [ ]:
final.head()

In [ ]:
fig, ax = plt.subplots(1, figsize=(50,20))

final.plot(edgecolor="black", ax=ax, alpha=0.3)

In [ ]:
# # Save shapefile
# final.to_file("./output/counties_NEX_intersect.shp")

## Combining with raster data (growing area)

In [ ]:
# County NEX intersection
counties = gp.read_file("./output/counties_NEX_intersect.shp")

In [ ]:
# Area fractions
fracs = gp.read_file("../../raster/output/area_fracs.shp")

In [ ]:
# Combine
final = gp.overlay(counties, fracs, how="intersection")

In [ ]:
# Add areas
final["area"] = final.geometry.area

In [ ]:
# There are some spurious intersections with extremely small areas... filter them out by choosing a filter that seems best
print(len(final))
print(len(final[(final.geometry.area > 10e-11)]))
print(len(final[(final.geometry.area > 10e-10)]))
print(len(final[(final.geometry.area > 10e-9)]))
print(len(final[(final.geometry.area > 10e-8)]))
print(len(final[(final.geometry.area > 10e-7)]))
print(len(final[(final.geometry.area > 10e-6)]))
print(len(final[(final.geometry.area > 10e-5)]))
print(len(final[(final.geometry.area > 10e-4)]))
print(len(final[(final.geometry.area > 10e-3)]))

In [ ]:
# Choose 10^-11
final = final[(final.geometry.area > 10e-11)]

In [ ]:
# Add county areas 
final = pd.merge(final, final.groupby("GEOID",as_index=False).sum().drop(columns = ["latitude", "longitude", "frac"]), on = "GEOID", how = "outer")

In [ ]:
final.head()

In [ ]:
# Calculate area fractions of counties
final["area_frac"] = final["area_x"] / final["area_y"]

del final["area_x"]
del final["area_y"]

In [ ]:
final.head()

In [ ]:
# Calculate weights within a signle county
final["county_weight"] = final["frac"] * final["area_frac"]

del final["frac"]
del final["area_frac"]

In [ ]:
# Calculate normalisations
final = pd.merge(final, final.groupby("GEOID",as_index=False).sum().drop(columns = ["latitude", "longitude"]), on = "GEOID", how = "outer")

In [ ]:
# Normalise
final["within_county_weight"] = final["county_weight_x"] / final["county_weight_y"]

del final["county_weight_x"]
del final["county_weight_y"]

final = final.fillna(0)

In [ ]:
final.head()

In [ ]:
# Check
final[final.GEOID == "17019"]["within_county_weight"].sum()

In [ ]:
# Check Champaign
fig, ax = plt.subplots(1, figsize=(10,10))

final[(final.GEOID == "17019")].plot(ax = ax, edgecolor = "gray", column = "within_county_weight", cmap = "Greys", legend = True)

In [ ]:
# Save
final.to_file("./output/counties_NEX_weighted.shp")

In [ ]:
# Save csv
del final["geometry"]
del final["NAME"]
final = pd.DataFrame(final)

final.to_csv("./output/counties_NEX_weighted.csv", index = False)